<a href="https://colab.research.google.com/github/ANITHAKISHORE/EDA/blob/main/Internship_Assignment_for_NLP(Voice_AI)_Creation_of_for_Speech_to_Text_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip3 install SpeechRecognition

In [ ]:
!pip3 install Transformers

In [ ]:
! pip3 install datasets

In [ ]:
!pip3 install accelerate

In [ ]:
!pip3 install whisper

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
print(output)

In [ ]:
len(output[1][0])

768

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32


In [ ]:
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, use_safetensors=True
)

In [ ]:
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps="word",
    torch_dtype=torch_dtype,
    device=device,
)

In [ ]:
import datasets

dataset = datasets.load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])

MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL NOR IS MISTER QUILTER'S MANNER LESS INTERESTING THAN HIS MATTER HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMS AND ROAST BEEF LOOMING BEFORE US SIMILES DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND HE HAS GRAVE DOUBTS WHETHER SIR FREDERICK LAYTON'S WORK IS READY GREEK AFTER ALL AND CAN DISCOVER IN IT BUT LITTLE OF ROCKY ITHACA LENELL'S PICTURES ARE A SORT OF UP GUARDS AND ADAM PAINTINGS AND MASON'S EXQUISITE IDLLS ARE AS NATIONAL AS A JINGO POEM MISTER BURKETT FOSTER'S LANDSCAPES SMILE AT ONE MUCH IN THE SAME WAY THAT MISTER CARKER USED TO FLASH HIS TEETH AND MISTER JOHN CAWLEYER GIVES HIS SIDER A CHEERFUL SLAP ON THE BACK BEFORE HE SAYS LIKE A SHAMPOOER IN A TURKISH BATH NEXT MAN


In [ ]:
sample = '/content/common_voice_mr_27591986.wav'
result = pipe(sample)
print(result["text"])

YAR BY NATFULLY CUTIS WUL OBIE LA HOI


In [ ]:
sample = '/content/common_voice_mr_27591987.wav'
result = pipe(sample)
print(result["text"])

ANAK RECHIMA ABUNGO GOENI AS HIS FOUT LECON DENNI GILLI


In [ ]:
print(calculate_wer(sample, result["text"]))

1.0


In [ ]:
# Evaluation metrics for the ASR -- Automatic Speech Recognition is (WER) Word Error Rate, Character Error Rate (CER), Sentence Error Rate(SER)

In [ ]:
!pip3 install wer

In [ ]:
def calculate_wer(reference, result):
  """Calculates the word error rate (WER) between two strings.

  Args:
    reference: The reference string.
    result: The result string.

  Returns:
    The WER.
  """

  # Convert the strings to lists of words.
  reference_words = reference.split()
  result_words = result.split()

  # Calculate the number of errors.
  num_errors = 0
  for i in range(len(reference_words)):
    if reference_words[i] != result_words[i]:
      num_errors += 1

  # Calculate the WER.
  wer = num_errors / len(reference_words)

  return wer